# JEC/JER Exercises

## Visualize the JEC
Plot the jet quantities before and after JEC:

In [ ]:
# Loads the ROOT environment and style
import ROOT as r
from collections import OrderedDict
from Analysis.JMEDAS.tdrstyle_mod14 import *
from tqdm import tqdm_notebook
#from ipywidgets import FloatProgress
#from IPython.display import display

# Set the ROOT style
r.gROOT.Macro("$CMSSW_BASE/src/Analysis/JMEDAS/scripts/rootlogon.C")
setTDRStyle()

In [ ]:
r.enableJSVis()

In [ ]:
# Open a file
f = r.TFile("JECNtuple_MiniAOD.root")
# Look at its contents
f.ls()
# Change directories and look at its contents
f.cd("AK4PFCHSL1L2L3")
r.gDirectory.ls()
# Get the tree and look at its branches
tree = r.gDirectory.Get("t")
tree.Show(0)

In [ ]:
#Set some options up
ifilename = "JECNtuple_MiniAOD.root" # The name of the input ROOT file.
algsize   = "AK4" # The algorithm and size of the jet collections to get from the ntuple.
doPUPPI   = False # Plot the PFPuppi algorithm as well (if it exists in the ntuple)
drawFits  = False # Flag to draw, or not, the Gaussian fits to the response curves
drawLines = True  # Flag to draw, or not, the lines at the mean of the Gaussian fit
maxEvents = -1    # The maximum number of events in the tree to use (default=-1 is all events)

#Settings for each of the pads in the canvas
settingsTMP = {'response' : (1,0.0,2.0,0.0,0.12,"Response (p_{T}^{RECO}/p_{T}^{GEN})","a.u.",False),
               'pt'       : (2,10.0,1000.0,0.0,0.21,"p_{T}^{RECO}","a.u.",True),
               'eta'      : (3,-5,5,0.0,0.1,"#eta","a.u.",False),
               'phi'      : (4,-3.14159,3.14159,0.0,0.04,"#phi","a.u.",False)
               }
settings = OrderedDict(sorted(settingsTMP.items(), key=lambda x:x[1], reverse=False))

# Create and draw the canvas
frames = []
for f, s in enumerate(settings) :
    frame = r.TH1D()
    frames.append(frame)
    frames[f].GetXaxis().SetLimits(settings[s][1],settings[s][2])
    frames[f].GetYaxis().SetRangeUser(settings[s][3],settings[s][4])
    frames[f].GetXaxis().SetTitle(settings[s][5])
    frames[f].GetYaxis().SetTitle(settings[s][6])
    if settings[s][6] :
        frames[f].GetXaxis().SetMoreLogLabels()
        frames[f].GetXaxis().SetNoExponent()
c = tdrCanvasMultipad("c",frames,14,11,2,2)

# Open the ROOT file with the ntuple
f = r.TFile(ifilename)

# Histogram settings
alg_size = algsize
jetTypes = OrderedDict([("PF" , r.kBlack),("PFCHS" , r.kRed),("PFPUPPI" , r.kGreen)])
corrections = OrderedDict([("Uncorrected" , r.kDotted),("L1" , r.kDashed),("L1L2L3" , r.kSolid)])
hsettingsTMP = {'response' : (1,80,0,2),
                'pt'       : (2,200,0,1000),
                'eta'      : (3,50,-5,5),
                'phi'      : (4,50,-3.14159,3.14159),
                }
hsettings = OrderedDict(sorted(hsettingsTMP.items(), key=lambda x:x[1], reverse=False))
nsigma = 1.0

histograms = {}
fits = {}
lines = {}
legends = {}
for hs in hsettings:
    print "Doing the",hs,"pad ... "

    legname = "leg_"+hs
    legends[legname] = tdrLeg(0.65,0.50,0.9,0.9)
    legends[legname].SetTextSize(0.035)

    for jt in jetTypes:
        if not doPUPPI and (jt == "PFPuppi" or jt == "PFPUPPI"):
            continue
        if not f.GetDirectory(alg_size+jt+"L1L2L3"):
            continue

        tree = f.Get(alg_size+jt+"L1L2L3/t")

        for icor, cor in enumerate(corrections):
            # Create the histograms
            hname = "h"+alg_size+jt+"_"+cor+"_"+hs

            print "\tDoing the histogram",hname,"..."

            histograms[hname] = r.TH1D(hname,hname,hsettingsTMP[hs][1],hsettingsTMP[hs][2],hsettingsTMP[hs][3])

            # Fill the histograms
            for ievent, event in enumerate(tqdm_notebook(tree,desc="event loop")):
                #if ievent%10000==0: print "\t\tEvent",ievent,"..."
                if maxEvents>-1 and ievent > maxEvents:
                    continue
                for jet, pt_from_tree in enumerate(event.jtpt):
                    if cor == "Uncorrected":
                        pt_updated = (pt_from_tree*event.jtjec[jet][0].second)
                    elif cor == "L1":
                        pt_updated = (pt_from_tree*event.jtjec[jet][1].second)
                    else:
                        pt_updated = pt_from_tree

                    if event.refpt[jet]==0:
                        continue
                    rsp = pt_updated/event.refpt[jet]
                    #print "JEC Level: "+str(event.jtjec[jet][0].first)+"\tJEC Factor: "+str(event.jtjec[jet][0].second)+"\tOriginal pT: "+str(pt_from_tree)+"\tUncorrected pT: "+str(pt_from_tree/event.jtjec[jet][0].second)+"\tRef pT: "+str(event.refpt[jet])
                    if rsp<2.0 and rsp>0.0 and abs(event.jteta[jet])<1.3 and event.refdrjt[jet]<float(alg_size[-1:])/20:
                        if hs == "response":
                            histograms[hname].Fill(rsp)
                        elif hs == "pt":
                            histograms[hname].Fill(pt_updated)
                        elif hs == "eta":
                            histograms[hname].Fill(event.jteta[jet])
                        elif hs == "phi":
                            histograms[hname].Fill(event.jtphi[jet])

            # Normalize the histograms
            histograms[hname].Scale(1.0/histograms[hname].Integral())

            if hs == "response":
                # Fit a Gaussian to the response curves
                fname = "f"+alg_size+jt+"_"+cor+"_"+hs
                fits[fname] = r.TF1(fname,"gaus",histograms[hname].GetMean()-(nsigma*histograms[hname].GetRMS()),histograms[hname].GetMean()+(nsigma*histograms[hname].GetRMS()))
                fits[fname].SetParNames("N","#mu","#sigma")
                fits[fname].SetLineColor(jetTypes[jt]-icor)
                fits[fname].SetLineStyle(corrections[cor])
                histograms[hname].Fit(fits[fname],"RQ0")

                # Create lines based on the fits
                lname = "l"+alg_size+jt+"_"+cor+"_"+hs
                lines[lname] = r.TLine(fits[fname].GetParameter(1),settings["response"][3],fits[fname].GetParameter(1),settings["response"][4])
                lines[lname].SetLineColor(jetTypes[jt]-icor)
                lines[lname].SetLineStyle(corrections[cor])

            # Add entries to the legend
            legends[legname].AddEntry(histograms[hname],str(alg_size+jt+cor).replace("Uncorrected",""),"l")

            c.cd(hsettingsTMP[hs][0])
            tdrDraw(histograms[hname],"HIST",r.kNone,jetTypes[jt]-icor,corrections[cor],-1,0,0)

            if hs == "response":
                # Draw the fits
                if drawFits:
                    c.cd(hsettingsTMP[hs][0])
                    fits[fname].Draw("same")

                # Draw the lines
                if drawLines:
                    c.cd(hsettingsTMP[hs][0])
                    lines[lname].Draw("same")
            elif hs == "pt":
                r.gPad.SetLogx()

    #Draw the legend
    c.cd(hsettingsTMP[hs][0])
    legends[legname].Draw("same")
c.Update()
c.Draw()

<font color='red'>Question 7: Is the mean value of the jet pT response closer or further from unity? Is the most provable value (peak) of the jet pT response closer or further from unity? Thinking about what these corrections are doing, does this seem right to you?</font>

<font color='red'>Question 8: Should the $\eta$ and $\phi$ distributions change?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
No, the JEC only changes the $p_{T}$ and energy of the jet. It basically scales jet 4-momentum without changing the jet direction.
</details>

## JEC Uncertainties and JER (Uncertainties)
Now we are going to take a look at how the JEC uncertainties and JER (uncertainties) effect the response distribution.

In [ ]:
jetcollection = "AK4PFCHSL1L2L3" # The jet collections to get from the ntuple
nsigma        = 1.0   # The number of sigma from the histogram mean which is used to determine the fit range
doPUPPI       = False # Plot the PFPuppi algorithm as well (if it exists in the ntuple)
drawFits      = False # Flag to draw, or not, the Gaussian fits to the response curves
noDrawLines   = True  # Flag to draw, or not, the lines at the mean of the Gaussian fit
maxEvents     = -1    # The maximum number of events in the tree to use (default=-1 is all events)

# Filename suffixes
ifilenames = OrderedDict([("",r.kBlack),("_JESUncertaintyUp",r.kGray+2),("_JESUncertaintyDown",r.kGray+2),("_JER",r.kRed),("_JERUncertaintyUp",r.kRed+2),("_JERUncertaintyDown",r.kRed+2)])

# Create and draw the canvas
frame = r.TH1D()
frame.GetXaxis().SetLimits(0.6,1.6)
frame.GetYaxis().SetRangeUser(0.0,0.07)
frame.GetXaxis().SetTitle("Response (p_{T}^{RECO}/p_{T}^{GEN})")
frame.GetYaxis().SetTitle("a.u.")
c = tdrCanvas("c",frame,14,11,True)

leg = tdrLeg(0.6,0.50,0.85,0.9)
leg.SetTextSize(0.03)
histograms = {}
fits = {}
lines = {}
for ifilename_suffix in ifilenames:
    ifilename = "JECNtuple_MiniAOD"+ifilename_suffix+".root"
    print "Processing the file "+ifilename+" ... "

    f = r.TFile(ifilename,"READ")

    if not f.GetDirectory(jetcollection):
        print "ERROR::Cannot find the jet collection "+jetcollection+" in the input file "+ifilename
        exit(-1)

    tree = f.Get(jetcollection+"/t")

    # Create the histograms
    hname = "h"+jetcollection+ifilename_suffix

    print "\tDoing the histogram",hname,"..."

    histograms[hname] = r.TH1D(hname,hname,80,0.6,1.6)
    histograms[hname].SetDirectory(None)

    # Fill the histograms
    for ievent, event in enumerate(tree):
        if maxEvents>-1 and ievent > maxEvents:
            continue
        for jet, pt_from_tree in enumerate(event.jtpt):
            pt_updated = pt_from_tree
            if event.refpt[jet]==0:
                continue
            rsp = pt_updated/event.refpt[jet]
            if rsp<2.0 and rsp>0.0 and abs(event.jteta[jet])<1.3:
                histograms[hname].Fill(rsp)

    # Normalize the histograms
    histograms[hname].Scale(1.0/histograms[hname].Integral())

    if "Up" in ifilename_suffix:
        style = r.kDotted
    elif "Down" in ifilename_suffix:
        style = r.kDashed
    else:
        style = r.kSolid

    # Fit a Gaussian to the response curves
    fname = "f"+jetcollection+ifilename_suffix
    fits[fname] = r.TF1(fname,"gaus",histograms[hname].GetMean()-(nsigma*histograms[hname].GetRMS()),histograms[hname].GetMean()+(nsigma*histograms[hname].GetRMS()))
    fits[fname].SetParNames("N","#mu","#sigma")
    fits[fname].SetLineColor(ifilenames[ifilename_suffix])
    fits[fname].SetLineStyle(style)
    histograms[hname].Fit(fits[fname],"RQ0")

    # Create lines based on the fits
    lname = "l"+jetcollection+ifilename_suffix
    lines[lname] = r.TLine(fits[fname].GetParameter(1),0.0,fits[fname].GetParameter(1),0.07)
    lines[lname].SetLineColor(ifilenames[ifilename_suffix])
    lines[lname].SetLineStyle(style)

    # Add entries to the legend
    leg.AddEntry(histograms[hname],ifilename_suffix.replace("_","") if ifilename_suffix!="" else str(jetcollection),"l")

    c.cd()
    tdrDraw(histograms[hname],"HIST",r.kNone,ifilenames[ifilename_suffix],style,-1,0,0)

    # Draw the fits
    if drawFits:
        fits[fname].Draw("same")

    # Draw the lines
    if drawLines:
        lines[lname].Draw("same")

#Draw the legend
leg.Draw("same")

c.Update()
c.Draw()

<font color='red'>Question 9: Is the JER distribution broader or less broad than the original distribution?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
It really depends on how we model jets in out MC samples. In our case our MC jets have too narrow a peak (a better resolution than we have in data). Thus we should be smearing out the distribution.
</details>

<font color='red'>Question 10: Do the up and down uncertainties look right? I.e. do they bracket the nominal distribution?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
They should.
</details>

## Get Text Files From A Global Tag
Here we will explain how to access the JEC text files for a given global tag. From here you can generalize this process to access other conditions (i.e. jet resolution) from the CMS databases.

First take a look at the code `jmehats_DBReader.py`. It's a very short piece of code, but the process can be generalized to get as many versions of the JEC as you'd like. We can run this code by using the command below.

In [ ]:
%%bash
cmsRun $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmehats_DBReader.py

You should now have files which have names like `94X_mc2017_realistic_v10_L1FastJet_AK4PFchs.txt`

<br>

<font color='red'>Question 11: How many files did you create?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
6
</details>

<br>

<font color='red'>Question 12: Go to https://cms-conddb.cern.ch/cmsDbBrowser and type '94X_mc2017_realistic_v10' into the search box. Click on the first link and then type 'JetCorrectionsRecord' into the search box. How many JEC records are there in this global tag? What is the era of the AK4PFchs correction?</font>

Note, this page may not load in Chrome. In Firefox you will have to click on "Advanced" to accept the certificate. This is fairly common for CERN sites. You will then authenticate with your grid certificate.<details>
<summary><font color='blue'>Show answer...</font></summary>
Count: 42
    
Era: Summer16_23Sep2016V4
</details>

<font color='red'>Question 13: Explore the CMS CondDB browser some more. What other information can you find?</font>

## On Your Own
Play around with the jmehats_JEC.py software. For instance try:
```bash
cmsRun jmehats_JEC.py print maxEvents=1000 doReclustering=1 doJetToolbox=1
```

<font color='red'>Question 14: Did that take more or less time to process all of the events than not using the JetToolbox?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
It will almost certainly take more time to recluster the jets than it would to just grab them from MiniAOD.
</details>

<br>

<font color='red'>Question 15: Can you add more jet collections?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
See ![lines 151-164](https://github.com/cms-jet/JMEDAS/blob/master/test/jmehats_JEC.py#L151-L164).
</details>

<br>

<font color='red'>Question 16: Are the jet collections equivalent if you only change the production method? i.e. Is AK4PFchs using the JetToolbox the same is AK4PFchs if you use the jet updater or addJetCollection?</font><details>
<summary><font color='blue'>Show answer...</font></summary>
These should all be equivalent methods. However, if you get the jets straight from MiniAOD and don't use the jet updater, and an old GT was used to make the MiniAOD, then you run the risk of your jets having outdated JEC.
</details>

<br>

<font color='red'>Maybe try using the legacy code. See what happens.</font>